In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
super_bowl_file = "Resources/Super_Bowl.csv"
super_bowl_df = pd.read_csv(super_bowl_file)
super_bowl_df.head()

,Date,SB,Attendance,QB Winner,Coach Winner,Winner,Winning Pts,QB Loser,Coach Loser,Loser,...,City,State,Point Difference,Referee,Umpire,Head Linesman,Line Judge,Field Judge,Back Judge,Side Judge
0,15-Jan-67,I,61946,Bart Starr,Vince Lombardi,Green Bay Packers,35,Len Dawson,Hank Stram,Kansas City Chiefs,...,Los Angeles,California,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14-Jan-68,II,75546,Bart Starr,Vince Lombardi,Green Bay Packers,33,Daryle Lamonica,John Rauch,Oakland Raiders,...,Miami,Florida,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12-Jan-69,III,75389,Joe Namath,Weeb Ewbank,New York Jets,16,"Earl Morrall, Johnny Unitas",Don Shula,Baltimore Colts,...,Miami,Florida,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11-Jan-70,IV,80562,"Len Dawson, Mike Livingston",Hank Stram,Kansas City Chiefs,23,Joe Kapp,Bud Grant,Minnesota Vikings,...,New Orleans,Louisiana,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17-Jan-71,V,79204,"Earl Morrall , Johnny Unitas",Don McCafferty,Baltimore Colts,16,Craig Morton,Tom Landry,Dallas Cowboys,...,Miami,Florida,3,Norm Schachter,Paul Trepinski,Ed Marion,Jack Fette,Fritz Graf,Hugh Gamber,NaN


In [ ]:
NYC_crime_file = "./Resources/NYPD_Complaint_Data_Historic.csv"
nyc_crime_df = pd.read_csv(NYC_crime_file)
nyc_crime_df.head()# Create a filtered dataframe from specific columns
nypd_cols = ["CMPLNT_NUM", "OFNS_DESC", "BORO_NM"]
new_nyc_crime_df= nyc_crime_df[nypd_cols].copy()

# Rename the column headers
new_nyc_crime_df = new_nyc_crime_df.rename(columns={"CMPLNT_NUM": "complaint_id",
                                                          "OFNS_DESC": "type_offense",
                                                          "BORO_NM": "borough_name"})

# Clean the data by dropping duplicates and setting the index
new_nyc_crime_df.drop_duplicates("complaint_id", inplace=True)
new_nyc_crime_df.set_index("complaint_id", inplace=True)

new_nyc_crime_df["borough_name"] = new_nyc_crime_df["borough_name"].astype(str).str.title()

new_nyc_crime_df.head()